In [1]:
sc

<SparkContext master=local[*] appName=PySparkShell>

In [2]:
taxi = sc.textFile('yellow.csv.gz')
bike = sc.textFile('citibike.csv')
list(enumerate(bike.take(1)[0].split(',')))

bike.take(2)

[u'cartodb_id,the_geom,tripduration,starttime,stoptime,start_station_id,start_station_name,start_station_latitude,start_station_longitude,end_station_id,end_station_name,end_station_latitude,end_station_longitude,bikeid,usertype,birth_year,gender',
 u'1,,801,2015-02-01 00:00:00+00,2015-02-01 00:14:00+00,521,8 Ave & W 31 St,40.75044999,-73.99481051,423,W 54 St & 9 Ave,40.76584941,-73.98690506,17131,Subscriber,1978,2']

In [3]:
def filterBike(pId, lines):
    import csv
    for row in csv.reader(lines):
        if (row[6] == 'Greenwich Ave & 8 Ave' and 
            row[3].startswith('2015-02-01')):
            yield (row[3][:19])
            
gBike = bike.mapPartitionsWithIndex(filterBike).cache()
gBike.take(5)

['2015-02-01 00:05:00',
 '2015-02-01 00:05:00',
 '2015-02-01 00:50:00',
 '2015-02-01 01:30:00',
 '2015-02-01 03:28:00']

In [4]:
bike.filter(lambda x: 'Greenwich Ave & 8 Ave' in x).first()

u'10,,585,2015-02-01 00:05:00+00,2015-02-01 00:15:00+00,284,Greenwich Ave & 8 Ave,40.73901691,-74.00263761,444,Broadway & W 24 St,40.7423543,-73.98915076,14843,Subscriber,1982,1'

In [5]:
def filterTaxi(pId, lines):
    if pId==0:
        next(lines)
    import csv
    for row in csv.reader(lines):
            yield row
            
gTaxi = taxi.mapPartitionsWithIndex(filterTaxi).cache()
gTaxi.take(5)

[['2015-02-01 00:00:00.0',
  '2015-02-01 00:10:10.0',
  '40.775485992431641',
  '-73.95855712890625',
  '40.720134735107422',
  '-73.9749755859375'],
 ['2015-02-01 00:00:00.0',
  '2015-02-01 00:03:36.0',
  '40.728248596191406',
  '-73.984840393066406',
  '40.731391906738281',
  '-73.975341796875'],
 ['2015-02-01 00:00:00.0',
  '2015-02-01 00:10:32.0',
  '40.731014251708984',
  '-74.001373291015625',
  '40.749237060546875',
  '-74.003067016601562'],
 ['2015-02-01 00:00:00.0',
  '2015-02-01 00:19:42.0',
  '40.721115112304688',
  '-73.987129211425781',
  '40.794017791748047',
  '-73.96990966796875'],
 ['2015-02-01 00:00:00.0',
  '2015-02-01 00:03:11.0',
  '40.756072998046875',
  '-73.983413696289062',
  '40.756237030029297',
  '-73.975303649902344']]

In [6]:
import pyproj

In [7]:
def filterTaxi(pId, lines):
    if pId==0:
        next(lines)
    import csv
    import pyproj
    proj = pyproj.Proj(init="epsg:2263", preserve_units=True)
    gLoc = proj(-74.00263761, 40.73901691)
    sqm = 1320**2
    for row in csv.reader(lines):
        try:
            dropOff = proj(float(row[5]), float(row[4]))
        except:
            continue
        sDistance = (dropOff[0]-gLoc[0])**2 + (dropOff[1]-gLoc[1])**2
        if sDistance<sqm: # 0.25 mile to feet 
            yield row[1][:19]
            
gTaxi = taxi.mapPartitionsWithIndex(filterTaxi).cache()
gTaxi.take(5)

['2015-02-01 00:11:03',
 '2015-02-01 00:10:23',
 '2015-02-01 00:16:36',
 '2015-02-01 00:10:14',
 '2015-02-01 00:10:12']

In [8]:
lBike = gBike.collect()
lTaxi = gTaxi.collect()

In [9]:
print(lTaxi[0])
print(lBike[0])

2015-02-01 00:11:03
2015-02-01 00:05:00


In [10]:
import datetime
count = 0
for b in lBike:
    bt = datetime.datetime.strptime(b, '%Y-%m-%d %H:%M:%S')
    for t in lTaxi:
        tt = datetime.datetime.strptime(t, '%Y-%m-%d %H:%M:%S')
        diff = (bt-tt).total_seconds()
        if diff>0 and diff<600:
            count += 1
            break
print(count)

65


In [11]:
gAll = gBike.map(lambda x: (x, 0)) + gBike.map(lambda x: (x, 1))   # concat two rdds
gAll.sortByKey().take(5)

[('2015-02-01 00:05:00', 0),
 ('2015-02-01 00:05:00', 0),
 ('2015-02-01 00:05:00', 1),
 ('2015-02-01 00:05:00', 1),
 ('2015-02-01 00:50:00', 0)]

In [12]:
def findTrips(_, records):
    import datetime
    for dt,event in records:
        t = datetime.datetime.strptime(b, '%Y-%m-%d %H:%M:%S')
        if event==1:
            if lastTaxiTime!=None:
                if (t-lastTaxiTime).total_seconds()<600:
                    yield (dt, event)
        else:
            lastTaxiTime = t
            
gAll.sortByKey().mapPartitionsWithIndex(findTrips).count()

65